In [3]:
import tensorflow as tf
import collections
import math
import os
import random
import zipfile

import numpy as np

In [4]:
with open('./text8', 'r') as f:
    words = tf.compat.as_str(f.read()).split()
print('Data size', len(words))



('Data size', 17005207)


In [5]:
# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 50000


def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])


('Most common words (+UNK)', [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)])
('Sample data', [5239, 3084, 12, 6, 195, 2, 3137, 46, 59, 156], ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against'])


In [6]:
dictionary

{'fawn': 45848,
 'homomorphism': 9648,
 'nordisk': 39343,
 'nunnery': 36075,
 'chthonic': 33554,
 'sowell': 40562,
 'sonja': 38175,
 'showa': 32906,
 'woods': 6263,
 'hsv': 44222,
 'spiders': 14623,
 'hanging': 8021,
 'woody': 11150,
 'comically': 38935,
 'localized': 16716,
 'schlegel': 39763,
 'sevens': 47931,
 'canes': 30965,
 'sprague': 19496,
 'chatter': 45028,
 'orthographies': 37085,
 'nanking': 40997,
 'originality': 19567,
 'mutinies': 47932,
 'alphabetic': 13925,
 'hermann': 6482,
 'cytochrome': 26060,
 'brocklin': 43516,
 'stipulate': 42056,
 'eugenics': 7421,
 'cofinality': 36724,
 'danmarks': 46453,
 'capoeira': 8971,
 'appropriation': 21010,
 'holyrood': 49046,
 'taj': 29360,
 'strictest': 30374,
 'bringing': 3627,
 'wooded': 18967,
 'liaisons': 38573,
 'vibrational': 25367,
 'umts': 33555,
 'wooden': 4779,
 'wednesday': 13043,
 'circuitry': 14992,
 'elgar': 16456,
 'stereotypical': 15093,
 'immunities': 38936,
 'pantheistic': 30327,
 'thrace': 12647,
 'kublai': 30769,
 '

In [7]:
data_index = 0

# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [skip_window]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)


In [8]:
[reverse_dictionary[i] for i in batch], [reverse_dictionary[i] for i in labels[:, 0]]
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

(3084, 'originated', '->', 5239, 'anarchism')
(3084, 'originated', '->', 12, 'as')
(12, 'as', '->', 3084, 'originated')
(12, 'as', '->', 6, 'a')
(6, 'a', '->', 12, 'as')
(6, 'a', '->', 195, 'term')
(195, 'term', '->', 2, 'of')
(195, 'term', '->', 6, 'a')


In [9]:
# Step 4: Build and train a skip-gram model.

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()

In [11]:
num_steps = 100001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print("Initialized")

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step ", step, ": ", average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = "Nearest to %s:" % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = "%s %s," % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()

Initialized
('Average loss at step ', 0, ': ', 271.43389892578125)
Nearest to first: greenery, unsatisfied, shit, priscus, objectivism, interred, patricia, outer,
Nearest to UNK: powerplant, sow, intergovernmentalism, counters, pentium, educated, tahiti, apical,
Nearest to has: benoit, spacecraft, paradise, compressors, circ, decry, mia, etext,
Nearest to used: modify, refreshing, decommissioned, sectors, collectivism, tidy, mattress, signifies,
Nearest to may: walras, dartmoor, santiago, defied, service, portfolios, noting, autos,
Nearest to so: funneled, sari, misgivings, hetzel, performances, redaction, estuary, hawaii,
Nearest to while: watterson, tilden, bearing, alam, patenting, electron, electronegative, ige,
Nearest to i: domingue, ghetto, ka, bcp, partita, mere, unlimited, outre,
Nearest to were: wisconsin, displayed, stagnant, ostia, succinctly, andropov, zhdanov, curiosity,
Nearest to by: spiegel, aqdas, netherland, metastasis, rediscovery, distinguishing, modulator, lowered

In [25]:
final_embeddings[dictionary['flow']]
import scipy.spatial 

print(final_embeddings[dictionary['flow']])
print(scipy.spatial.distance.cosine(final_embeddings[dictionary['flow']], final_embeddings[dictionary['flow']]))

print(scipy.spatial.distance.cosine(final_embeddings[dictionary['tensor']], final_embeddings[dictionary['flow']]))
print(scipy.spatial.distance.cosine(final_embeddings[dictionary['stream']], final_embeddings[dictionary['flow']]))
print(scipy.spatial.distance.cosine(final_embeddings[dictionary['red']], final_embeddings[dictionary['blue']]))

[ 0.1657137   0.03434748  0.06846502  0.0070382   0.0350267   0.03630972
  0.13870713 -0.08952355 -0.01397907  0.10646137  0.20203994 -0.02784512
  0.00405212  0.07034153 -0.08157731 -0.09949311 -0.06979465  0.07609912
  0.126651    0.1981343  -0.01641454  0.10250082  0.01758348 -0.04216308
 -0.15933873  0.07945434 -0.01796242 -0.00106697 -0.02517156 -0.12261134
  0.11137157  0.03639388 -0.1277702   0.00541975  0.07960305  0.03566069
 -0.04528071 -0.09769962  0.06677049 -0.02655403 -0.03063397  0.04496832
 -0.06530884  0.06109088 -0.04010385  0.01271805  0.05815612 -0.16572201
  0.00262617 -0.03960225 -0.09188902  0.14104255  0.12343952  0.08305998
 -0.04416177 -0.09624919 -0.09049979 -0.10931598  0.20486616 -0.00828655
  0.13724695  0.03226418  0.05047261  0.06022134  0.12492619  0.15182598
 -0.06894512 -0.02406714  0.07673304  0.02638871  0.02326527 -0.07070541
 -0.09337375 -0.08966452  0.16761334 -0.12091971 -0.02916223 -0.06994145
 -0.07964484 -0.01511631  0.03522532  0.06124158  0

In [32]:
def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
  assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
  plt.figure(figsize=(18, 18))  # in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

  plt.savefig(filename)

try:
  from sklearn.manifold import TSNE
  import matplotlib.pyplot as plt

  tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
  plot_only = 500
  low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
  labels = [reverse_dictionary[i] for i in xrange(plot_only)]
  plot_with_labels(low_dim_embs, labels)

except ImportError:
  print("Please install sklearn, matplotlib, and scipy to visualize embeddings.")